# Inferência — Queda Livre (Colab)

Notebook para carregar o modelo treinado (`model/fall_model`) gerado pelo `QuedaLivre_colab.ipynb` e fazer previsões a partir de tempos (t).

In [ ]:
# Célula 1 — montar Drive no Colab (opcional)
IN_COLAB = False
try:
    import google.colab
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    print('Drive montado. Copie a pasta model/ para /content/drive/MyDrive/... se quiser persistir o modelo.')
else:
    print('Executando localmente — verifique se a pasta model/ existe.')

In [ ]:
# Célula 2 — carregar modelo e scalers
import os
import numpy as np
from tensorflow.keras.models import load_model

MODEL_PATH = 'model/fall_model'
SCALER_PATH = 'model/scaler.npz'

if not os.path.exists(MODEL_PATH):
    print('Modelo não encontrado em', MODEL_PATH, '\nSe você estiver no Colab monte o Drive e copie a pasta model/ para este runtime, ou execute o notebook de treino.')
else:
    model = load_model(MODEL_PATH)
    sc = np.load(SCALER_PATH)
    t_mean, t_std = sc['t_mean'], sc['t_std']
    s_mean, s_std = sc['s_mean'], sc['s_std']
    print('Modelo e scalers carregados com sucesso.')

In [ ]:
# Célula 3 — função de predição interativa
import numpy as np
def predict_time(t_seconds):
    t_arr = np.array([[float(t_seconds)]], dtype=np.float32)
    t_n = (t_arr - t_mean) / t_std
    s_n = model.predict(t_n)
    s = s_n * s_std + s_mean
    return float(s[0,0])

# exemplo
if 'model' in globals():
    for t in [0.5, 1.0, 2.0]:
        print(t, 's ->', predict_time(t))
else:
    print('Modelo não carregado — execute a célula de carregamento primeiro.')

In [ ]:
# Célula 4 — inferência em lote via upload CSV (coluna com tempos em segundos)
try:
    from google.colab import files
    uploaded = files.upload()
    for name in uploaded:
        import io
        import pandas as pd
        df = pd.read_csv(io.BytesIO(uploaded[name]))
        if df.shape[1] == 1:
            times = df.iloc[:,0].values
        elif 't' in df.columns:
            times = df['t'].values
        else:
            raise ValueError('CSV deve ter coluna única com tempos ou coluna chamada t')
        preds = [predict_time(float(x)) for x in times]
        out_df = pd.DataFrame({'t': times, 's_pred': preds})
        out_name = 'predictions_' + name
        out_df.to_csv(out_name, index=False)
        files.download(out_name)
except Exception as e:
    print('Upload em lote não disponível ou falhou:', e)

---
### Dicas
- Se quiser salvar o resultado no Drive, copie o arquivo `predictions_*.csv` para uma pasta dentro de `/content/drive/MyDrive/`.
- Para adaptar o notebook para estimar tempo a partir de distância, carregue um modelo treinado nessa tarefa ou gere um dataset invertido no notebook de treino.